In [1]:
import numpy as np
import time
import logging
import ipywidgets as widgets
from IPython.display import clear_output, display
from module.print import printTable, printNode
from module.node import Node
from module.generate_children import get_all_children
from module.sortNode import sortNode
from module.createBoatActions import boat_actions

def logging_file(open_list, close_list):
    logging.basicConfig(filename= "basic.log", level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

    logging.info('********open list********, len = {}'.format(len(open_list)))
    for open_data in open_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   open_data.state,
                                                                                    open_data.data[0], open_data.data[1], open_data.data[2], 
                                                                                    open_data.boatA, open_data.boatB))
    logging.info('********close list********, len = {}'.format(len(close_list)))
    for close_data in close_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   close_data.state,
                                                                                    close_data.data[0], close_data.data[1], close_data.data[2], 
                                                                                    close_data.boatA, close_data.boatB))
    logging.info('--------------------------------------------------------------------------------------------------------')

def calculate_path(curr_node):
    result = []
    result.append(curr_node)
    parent_node = curr_node.parent
    while (parent_node):
        result.append(parent_node)
        parent_node = parent_node.parent
    return result

def uniform_cost_search(goal, start_node, setting, boatSetting, limit):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 0
    count_open = 1
    count_close = 0
    start_time = time.time()
    
    while True:
        if not open_list:
            print ("no solution!!! --- open list is empty\n")

            with open('table.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break
        if (time.time() - start_time > 90):
            print ("no solution!!! --- timeout (exceed 1 min)\n")

            with open('table.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break
        
        curr_node = open_list.pop()

        if (curr_node.state['m'] == goal[0] and curr_node.state['c'] == goal[1]): # find goal
            
            result = calculate_path(curr_node)
            printTable(result, setting)
            print(f"count_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            with open('table.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break

        close_list.append(curr_node)
        count_close = count_close + 1
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar, limit) # cost: 0 , time: 1
        
        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0
            
            for i in range(len(close_list)):
                if np.array_equal(child.state, close_list[i].state) and np.array_equal(child.data, close_list[i].data):
                    sameNode_in_close_list = 1
                    break
            
            for j in range(len(open_list)):
                if np.array_equal(child.state, open_list[j].state):
                    sameNode_in_open_list = 1
                    if child.data[ setting['costOrStep'] ] < open_list[j].data[ setting['costOrStep'] ]:
                        open_list.pop(j)
                    break
            
            if (sameNode_in_close_list != 1 and sameNode_in_open_list != 1):
                open_list.append(child)
                count_open =  1 + count_open
        sortNode(open_list, setting['costOrStep'], useAStar)
        #logging_file(open_list, close_list)

def AStar_algorithm(goal, start_node, setting, boatSetting, limit):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 1
    count_open = 1
    count_close = 0

    while True:
        if not open_list:
            print ("no solution!!! --- open_list is empty\n")
            
            with open('table.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break
        
        curr_node = open_list.pop()
        
        if (curr_node.state['m'] == goal[0] and curr_node.state['c'] == goal[1] ):
            
            result = calculate_path(curr_node)
            printTable(result, setting)
            print(f"count_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")

            with open('table.txt', 'a') as w:
                w.write(f"\ncount_open: {count_open}, count_close: {count_close}, open+close: {count_open + count_close}")
            break

        close_list.append(curr_node)
        count_close = count_close + 1
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar, limit) 

        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0

            for i in range(len(open_list)):
                if np.array_equal(child.state, open_list[i].state):
                    if child.data[2] < open_list[i].data[2]: # beter => remove older, append newer, AStar_f
                        open_list.pop(i)
                    else : # newer no better
                        sameNode_in_open_list = 1
                    break

            if (sameNode_in_open_list != 1):
                for j in range(len(close_list)):
                    if np.array_equal(child.state, close_list[j].state):
                        if child.data[2] < close_list[j].data[2]: # beter => remove older, append newer, AStar_f
                            close_list.pop(j)
                        else : # newer no better
                            sameNode_in_close_list = 1
                        break

                if (sameNode_in_close_list != 1):
                    open_list.append(child)
                    count_open =  1 + count_open

        sortNode(open_list, setting['costOrStep'], useAStar)
        #logging_file(open_list, close_list)

boatSetting = dict()
boatSetting['A'] = { 'capacity': 2, 'cost': 3,  'step' : 1, 'pos' : 1} # pos : right bank: 1, left bank: -1
boatSetting['B'] = { 'capacity': 3, 'cost': 25, 'step' : 1, 'pos' : 1}
boatA_operations = boat_actions('A', boatSetting)
boatB_operations = boat_actions('B', boatSetting)

limit = dict()
limit['cost'] = {'max': 1000}
limit['step'] = {'max': 100}

totalM = 3
totalC = 3
setting = {'totalM': totalM, 'totalC' : totalC, 'costOrStep' : 0}
start_node = Node(setting['totalM'], setting['totalC'], 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
goal_node = [0, 0]

def control_costOrStep():
    dropdown = widgets.Dropdown(
    options=[('cost', 0), ('step', 1)],
    value= None,
    description='cost or step: ',
)
    def on_dropdown_change(change):
        with out:
            clear_output()

            global setting
            setting['costOrStep'] = change.new
            start_node = Node(setting['totalM'], setting['totalC'], 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態
            
            start_time = time.time()
            uniform_cost_search(goal_node, start_node, setting, boatSetting, limit)
            print(f"execution time: {round(time.time() - start_time, 3)}")
        
    dropdown.observe(on_dropdown_change, names="value")   
    return dropdown

def control_num_missionary_cannibal(description, minValue, maxValue, value):
    intRangeSlider = widgets.IntSlider(
        value = value,
        min= minValue,
        max= maxValue,
        step=1,
        description = description,
        disabled = False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout= widgets.Layout(width='30%', height='45%')
    )
    
    def on_intRangeSlider_change(change):
        with out:
            clear_output()
            print(description, change.new)

            global setting
            if description == 'totalM' or description == 'totalC':
                setting[description] = change.new
                
            elif description == 'cost' or description == 'step':
                global limit
                setting['costOrStep'] = 1  if (description == 'cost') else 0
                limit[description]['max'] = change.new
                
            start_node = Node(setting['totalM'], setting['totalC'], 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
            
            start_time = time.time()
            uniform_cost_search(goal_node, start_node, setting, boatSetting, limit)
            print(f"execution time: {round(time.time() - start_time, 3)}")
    
    intRangeSlider.observe(on_intRangeSlider_change, names="value") 
    
    return intRangeSlider
    


person_widgets = widgets.HBox([
                control_costOrStep(),
                control_num_missionary_cannibal('totalM', 0, 15, 3), 
                control_num_missionary_cannibal('totalC', 0, 15, 3),
            ])
limit_widgets = widgets.HBox([
                control_num_missionary_cannibal('cost', 0, 1000, 500),
                control_num_missionary_cannibal('step', 0, 100, 20)
            ])

display(person_widgets)
display(limit_widgets)

out = widgets.Output()
display(out)

boatA 有6種動作，各為: [[0, 0, 3, 1], [0, 1, 3, 1], [0, 2, 3, 1], [1, 0, 3, 1], [1, 1, 3, 1], [2, 0, 3, 1]]

boatB 有9種動作，各為: [[0, 0, 25, 1], [0, 1, 25, 1], [0, 2, 25, 1], [0, 3, 25, 1], [1, 0, 25, 1], [1, 1, 25, 1], [2, 0, 25, 1], [2, 1, 25, 1], [3, 0, 25, 1]]



Output()